In [33]:
import numpy as np
import pandas as pd
from tensorflow.python.framework import ops
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [3]:
y_hat = tf.constant(36, name='y_hat')            # Define y_hat constant. Set to 36.
y = tf.constant(39, name='y')                    # Define y. Set to 39

loss = tf.Variable((y - y_hat)**2, name='loss')  # Create a variable for the loss

init = tf.global_variables_initializer()         # When init is run later (session.run(init)),
                                                 # the loss variable will be initialized and ready to be computed
with tf.Session() as session:                    # Create a session and print the output
    session.run(init)                            # Initializes the variables
    print(session.run(loss))                     # Prints the loss

Instructions for updating:
Colocations handled automatically by placer.
9


In [34]:
scaler = MinMaxScaler()

In [4]:
data = pd.read_csv("train.csv")

In [5]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
col = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
x_train = data[col]
y_train = data['Survived']

In [7]:
def format(data):
    data.drop(['Cabin', 'Name', 'Ticket'], axis = 1, inplace = True)
    data.fillna(data.mean(), inplace = True)
    data['Sex'] = data['Sex'].map({'male':1, 'female':0})
    data['Embarked'] = data['Embarked'].apply(lambda x : str(x))
    data['Embarked'] = data['Embarked'].map({'S':1, 'C':2, 'Q':3, 'nan' : 1})
    return data

In [8]:
train_x = format(x_train.copy())

In [9]:
train_x.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,1,22.0,1,0,7.2500,1
1,2,1,0,38.0,1,0,71.2833,2
2,3,3,0,26.0,0,0,7.9250,1
3,4,1,0,35.0,1,0,53.1000,1
4,5,3,1,35.0,0,0,8.0500,1


In [10]:
train_y = np.array([y_train, 1-y_train], dtype = float)
train_y.shape

(2, 891)

In [36]:
X_train = np.array(train_x, dtype = float).T
Y_train = np.array([y_train, 1-y_train], dtype = float)

In [37]:
X_train

array([[  1.    ,   2.    ,   3.    , ..., 889.    , 890.    , 891.    ],
       [  3.    ,   1.    ,   3.    , ...,   3.    ,   1.    ,   3.    ],
       [  1.    ,   0.    ,   0.    , ...,   0.    ,   1.    ,   1.    ],
       ...,
       [  0.    ,   0.    ,   0.    , ...,   2.    ,   0.    ,   0.    ],
       [  7.25  ,  71.2833,   7.925 , ...,  23.45  ,  30.    ,   7.75  ],
       [  1.    ,   2.    ,   1.    , ...,   1.    ,   2.    ,   3.    ]])

In [38]:
X_train = scaler.fit_transform(X_train)
Y_train = scaler.fit_transform(Y_train)

In [56]:
X_train

array([[0.04545455, 0.02805706, 0.11538462, ..., 1.        , 1.        ,
        1.        ],
       [0.13636364, 0.01402853, 0.11538462, ..., 0.00337458, 0.0011236 ,
        0.003367  ],
       [0.04545455, 0.        , 0.        , ..., 0.        , 0.0011236 ,
        0.00112233],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00224972, 0.        ,
        0.        ],
       [0.32954545, 1.        , 0.30480769, ..., 0.02637795, 0.03370787,
        0.00869809],
       [0.04545455, 0.02805706, 0.03846154, ..., 0.00112486, 0.00224719,
        0.003367  ]])

In [40]:
print(X_train.shape)
print(Y_train.shape)

(8, 891)
(2, 891)


In [41]:
def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, [n_x, None], name = "X")
    Y = tf.placeholder(tf.float32, [n_y, None], name = "Y")
    return X, Y

In [186]:
def initialize_parameters(n_x, n_y):
    W1 = tf.get_variable("W1", [8, n_x], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    W2 = tf.get_variable("W2", [4, 8], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    W3 = tf.get_variable("W3", [n_y, 4], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
#     W4 = tf.get_variable("W4", [n_y, 100], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b1 = tf.get_variable("b1", [8, 1], initializer = tf.zeros_initializer())
    b2 = tf.get_variable("b2", [4, 1], initializer = tf.zeros_initializer())
    b3 = tf.get_variable("b3", [n_y, 1], initializer = tf.zeros_initializer())
#     b4 = tf.get_variable("b4", [n_y, 1], initializer = tf.zeros_initializer())
    parameters = {"W1" : W1,
                  "W2" : W2,
                  "W3" : W3,
#                   "W4" : W4,
                  "b1" : b1,
                  "b2" : b2,
                  "b3" : b3}
#                   "b4" : b4}
    return parameters

In [214]:
def forward_propagation(X, parameters):
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
#     W4 = parameters["W4"]
    b1 = parameters["b1"]
    b2 = parameters["b2"]
    b3 = parameters["b3"]
#     b4 = parameters["b4"]
#     regularizer = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3)
    Z1 = tf.add(tf.matmul(W1, X), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)
    A2_ = tf.nn.relu(Z2)
#     keep_prob = tf.placeholder(tf.float32)
    A2 = tf.nn.dropout(A2_, 0.5)
    Z3 = tf.add(tf.matmul(W3, A2), b3)
#     A3 = tf.nn.relu(Z3)
#     Z4 = tf.add(tf.matmul(W4, A3), b4)
    
    return Z3#, regularizer

In [188]:
def compute_cost(Z3, Y):
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
#     a = tf.cast(tf.sigmoid(Z3) >= 0, "float")
#     b = Y
#     m = Y.shape[1]
#     cost = -1 * (tf.reduce_sum((tf.matmul(b, tf.log(tf.transpose(a)))) + (tf.matmul(1 - b, tf.log(tf.transpose(1 - a)))))) / m
    return cost

In [215]:
def model(X_train, Y_train, learning_rate = 0.003, num_epoches = 4000, beta = 0.01):
    ops.reset_default_graph()
    (n_x, m) = X_train.shape
    n_y = Y_train.shape[0]
    costs = []
    
    X, Y = create_placeholders(n_x, n_y)
    parameters = initialize_parameters(n_x, n_y)
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
#     cost = tf.reduce_mean(cost + beta*regularizer)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(num_epoches):
            _, epoch_cost = sess.run([optimizer, cost], feed_dict = {X : X_train, Y : Y_train})
            if epoch%5 == 0:
                costs.append(epoch_cost)
            if epoch%100 == 0:
                print("Cost after ", epoch, "epoch = ", epoch_cost)
    
#         print(parameters)
        parameters = sess.run(parameters)
#         print(parameters)
#         print(sess.run(tf.sigmoid(Z3) > 0.5, feed_dict = {X : X_train}))
        print(sess.run(Y, feed_dict = {Y : Y_train}))
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print("Train Accuracy : ", accuracy.eval({X : X_train, Y : Y_train}))

        return parameters

In [216]:
# Y_train

In [217]:
parameters = model(X_train, Y_train)

Cost after  0 epoch =  0.7108289
Cost after  100 epoch =  0.66205007
Cost after  200 epoch =  0.6578398
Cost after  300 epoch =  0.63779444
Cost after  400 epoch =  0.6254576
Cost after  500 epoch =  0.6103406
Cost after  600 epoch =  0.61400187
Cost after  700 epoch =  0.603712
Cost after  800 epoch =  0.5943104
Cost after  900 epoch =  0.58429086
Cost after  1000 epoch =  0.5789651
Cost after  1100 epoch =  0.5728004
Cost after  1200 epoch =  0.57220423
Cost after  1300 epoch =  0.5781846
Cost after  1400 epoch =  0.5800658
Cost after  1500 epoch =  0.57516414
Cost after  1600 epoch =  0.5679616
Cost after  1700 epoch =  0.5515233
Cost after  1800 epoch =  0.5584872
Cost after  1900 epoch =  0.5630969
Cost after  2000 epoch =  0.55595416
Cost after  2100 epoch =  0.5589169
Cost after  2200 epoch =  0.54358935
Cost after  2300 epoch =  0.5641276
Cost after  2400 epoch =  0.5574869
Cost after  2500 epoch =  0.5567837
Cost after  2600 epoch =  0.55456734
Cost after  2700 epoch =  0.5446

In [118]:
test_data = pd.read_csv("test.csv")

In [119]:
test_x = format(test_data.copy())

In [120]:
test_x.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,3
1,893,3,0,47.0,1,0,7.0000,1
2,894,2,1,62.0,0,0,9.6875,3
3,895,3,1,27.0,0,0,8.6625,1
4,896,3,0,22.0,1,1,12.2875,1


In [121]:
X_test = np.array(test_x).T

In [122]:
X_test = scaler.fit_transform(X_test)

In [123]:
X_test.shape

(8, 418)

In [124]:
# parameters

In [218]:
W1 = tf.convert_to_tensor(parameters["W1"])
b1 = tf.convert_to_tensor(parameters["b1"])
W2 = tf.convert_to_tensor(parameters["W2"])
b2 = tf.convert_to_tensor(parameters["b2"])
W3 = tf.convert_to_tensor(parameters["W3"])
b3 = tf.convert_to_tensor(parameters["b3"])

params = {"W1": W1,
          "b1": b1,
          "W2": W2,
          "b2": b2,
          "W3": W3,
          "b3": b3}
# print(params)
x = tf.placeholder(tf.float32, [X_train.shape[0], X_train.shape[1]])
pred = forward_propagation(x, params)
pred1 = tf.argmax(pred)
with tf.Session() as sess:
    predictions = sess.run(pred1, feed_dict = {x : X_train})
    print(predictions)

[1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 1 0 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1
 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0
 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 0 1 1 1 

In [219]:
ans = pd.DataFrame([test_x["PassengerId"], predictions]).T
ans.columns = ["PassengerID", "Survived"]

In [220]:
ans.head()

,PassengerID,Survived
0,892,1
1,893,0
2,894,0
3,895,1
4,896,1


In [221]:
ans.to_csv("ans.csv", index = False)